# HVAC optimization case study
## Feature engineering

### Setup

In [12]:
from pathlib import Path
import pandas as pd

In [13]:
root_path = Path.cwd().parent
interim_data_path = root_path.joinpath("data/interim")
processed_data_path = root_path.joinpath("data/processed")

### Load data

In [14]:
train_df = pd.read_csv(interim_data_path.joinpath("train_interim.csv"), index_col=0, parse_dates=True)
train_df.name = "train"
test_df = pd.read_csv(interim_data_path.joinpath("test_interim.csv"), index_col=0, parse_dates=True)
test_df.name = "test"
valid_df = pd.read_csv(interim_data_path.joinpath("valid_interim.csv"), index_col=0, parse_dates=True)
valid_df.name = "valid"
dataframes = [train_df, test_df, valid_df]

In [15]:
train_df.head()

,temperature,humidity,light,co2,humidityratio,occupancy
date,,,,,,
2015-02-04 17:51:00,23.18,27.2720,426.0,721.25,0.004793,1
2015-02-04 17:52:00,23.15,27.2675,429.5,714.00,0.004783,1
2015-02-04 17:53:00,23.15,27.2450,426.0,713.50,0.004779,1
2015-02-04 17:54:00,23.15,27.2000,426.0,708.25,0.004772,1
2015-02-04 17:55:00,23.10,27.2000,426.0,704.50,0.004757,1


In [16]:
test_df.tail()

,temperature,humidity,light,co2,humidityratio,occupancy
date,,,,,,
2015-02-04 10:39:00,24.290000,25.700000,808.0,1150.25,0.004829,1
2015-02-04 10:40:00,24.330000,25.736000,809.8,1129.20,0.004848,1
2015-02-04 10:41:00,24.330000,25.700000,817.0,1125.80,0.004841,1
2015-02-04 10:42:00,24.356667,25.700000,813.0,1123.00,0.004849,1
2015-02-04 10:43:00,24.408333,25.681667,798.0,1124.00,0.004860,1


In [17]:
valid_df.head()

,temperature,humidity,light,co2,humidityratio,occupancy
date,,,,,,
2015-02-11 14:48:00,21.7600,31.133333,437.333333,1029.666667,0.005021,1
2015-02-11 14:49:00,21.7900,31.000000,437.333333,1000.000000,0.005009,1
2015-02-11 14:50:00,21.7675,31.122500,434.000000,1003.750000,0.005022,1
2015-02-11 14:51:00,21.7675,31.122500,439.000000,1009.500000,0.005022,1
2015-02-11 14:52:00,21.7900,31.133333,437.333333,1005.666667,0.005030,1


### Add features
Cutoff values for CO2 and light are based on the following information:
* https://www.kane.co.uk/knowledge-centre/what-are-safe-levels-of-co-and-co2-in-rooms
* https://www.hse.gov.uk/temperature/thermal/factors.htm
* https://www.hse.gov.uk/humanfactors/topics/lighting.htm

In [18]:
for df in dataframes:
    df["year"] = df.index.year
    df["month"] = df.index.month
    df["day"] = df.index.day
    df["weekday"] = df.index.weekday
    df["hour"] = df.index.hour
    df["minute"] = df.index.minute
    df["week"] = df.index.weekofyear
    df["weekend"] = (df["weekday"] == 5) | (df["weekday"] == 6) # weekday starts with Mon = 0
    df["office_hour"] = (df["hour"] >= 9) & (df["hour"] < 17)
    df["lunch_break"] = (df["hour"] >= 12) & (df["hour"] < 13)
    df["night"] = (df["hour"] >= 19) & (df["hour"] < 6)
    df["high_co2"] = df["co2"] >= 1000
    df["light_on"] = df["light"] >= 50

In [19]:
train_df.head()

,temperature,humidity,light,co2,humidityratio,occupancy,year,month,day,weekday,hour,minute,week,weekend,office_hour,lunch_break,night,high_co2,light_on
date,,,,,,,,,,,,,,,,,,,
2015-02-04 17:51:00,23.18,27.2720,426.0,721.25,0.004793,1,2015,2,4,2,17,51,6,False,False,False,False,False,True
2015-02-04 17:52:00,23.15,27.2675,429.5,714.00,0.004783,1,2015,2,4,2,17,52,6,False,False,False,False,False,True
2015-02-04 17:53:00,23.15,27.2450,426.0,713.50,0.004779,1,2015,2,4,2,17,53,6,False,False,False,False,False,True
2015-02-04 17:54:00,23.15,27.2000,426.0,708.25,0.004772,1,2015,2,4,2,17,54,6,False,False,False,False,False,True
2015-02-04 17:55:00,23.10,27.2000,426.0,704.50,0.004757,1,2015,2,4,2,17,55,6,False,False,False,False,False,True


In [20]:
test_df.tail()

,temperature,humidity,light,co2,humidityratio,occupancy,year,month,day,weekday,hour,minute,week,weekend,office_hour,lunch_break,night,high_co2,light_on
date,,,,,,,,,,,,,,,,,,,
2015-02-04 10:39:00,24.290000,25.700000,808.0,1150.25,0.004829,1,2015,2,4,2,10,39,6,False,True,False,False,True,True
2015-02-04 10:40:00,24.330000,25.736000,809.8,1129.20,0.004848,1,2015,2,4,2,10,40,6,False,True,False,False,True,True
2015-02-04 10:41:00,24.330000,25.700000,817.0,1125.80,0.004841,1,2015,2,4,2,10,41,6,False,True,False,False,True,True
2015-02-04 10:42:00,24.356667,25.700000,813.0,1123.00,0.004849,1,2015,2,4,2,10,42,6,False,True,False,False,True,True
2015-02-04 10:43:00,24.408333,25.681667,798.0,1124.00,0.004860,1,2015,2,4,2,10,43,6,False,True,False,False,True,True


In [21]:
valid_df.head()

,temperature,humidity,light,co2,humidityratio,occupancy,year,month,day,weekday,hour,minute,week,weekend,office_hour,lunch_break,night,high_co2,light_on
date,,,,,,,,,,,,,,,,,,,
2015-02-11 14:48:00,21.7600,31.133333,437.333333,1029.666667,0.005021,1,2015,2,11,2,14,48,7,False,True,False,False,True,True
2015-02-11 14:49:00,21.7900,31.000000,437.333333,1000.000000,0.005009,1,2015,2,11,2,14,49,7,False,True,False,False,True,True
2015-02-11 14:50:00,21.7675,31.122500,434.000000,1003.750000,0.005022,1,2015,2,11,2,14,50,7,False,True,False,False,True,True
2015-02-11 14:51:00,21.7675,31.122500,439.000000,1009.500000,0.005022,1,2015,2,11,2,14,51,7,False,True,False,False,True,True
2015-02-11 14:52:00,21.7900,31.133333,437.333333,1005.666667,0.005030,1,2015,2,11,2,14,52,7,False,True,False,False,True,True


### Save transformed data

In [22]:
for df in dataframes:
    df.to_csv(processed_data_path.joinpath(f"{df.name}_processed.csv"), index=False)

In [23]:
!ls -alh $processed_data_path

total 4392
drwxr-xr-x  6 stefan  staff   192B Mar 27 15:24 .
drwxr-xr-x  6 stefan  staff   192B Mar  9 12:48 ..
-rw-r--r--@ 1 stefan  staff   6.0K Mar 27 15:24 .DS_Store
-rw-r--r--  1 stefan  staff   269K Mar 27 15:46 test_processed.csv
-rw-r--r--  1 stefan  staff   807K Mar 27 15:46 train_processed.csv
-rw-r--r--  1 stefan  staff   979K Mar 27 15:46 valid_processed.csv
